In [2]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade --force-reinstall numpy==1.26.4 scikit-learn==1.5.2


  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 35.4/35.4 MB 68.9 MB/s eta 0:00:01

In [3]:
# 2. Imports
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from tqdm import tqdm
import joblib

In [8]:
# 3. Configuration
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/train'  # Adjust to your mounting path
IMG_SIZE = (128, 128)
HOG_PARAMS = {
    'pixels_per_cell': (16, 16),
    'cells_per_block': (2, 2),
    'orientations': 9,
    'block_norm': 'L2-Hys'
}

In [9]:
# 4. Load and preprocess images, extract HOG features
def load_data(data_path):
    features = []
    labels = []
    for fname in tqdm(os.listdir(data_path)):
        label = None
        if fname.lower().startswith('cat'):
            label = 0
        elif fname.lower().startswith('dog'):
            label = 1
        else:
            continue
        try:
            img = imread(os.path.join(data_path, fname))
            img = resize(img, IMG_SIZE, anti_aliasing=True)
            if img.ndim == 3:
                img = np.mean(img, axis=2)  # Convert to grayscale
            hog_feat = hog(img, **HOG_PARAMS)
            features.append(hog_feat)
            labels.append(label)
        except Exception as e:
            print(f"Could not process image {fname}: {e}")
    return np.array(features), np.array(labels)

X, y = load_data(DATA_PATH)
print("Features shape:", X.shape, "Labels shape:", y.shape)


100%|██████████| 25000/25000 [18:59<00:00, 21.93it/s]


Features shape: (25000, 1764) Labels shape: (25000,)


In [10]:
# 5. Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [11]:
# 6. Train SVM
print("Training SVM—this may take a while...")
svm_clf = SVC(kernel='linear', C=1.0, probability=True, random_state=42)
svm_clf.fit(X_train, y_train)

Training SVM—this may take a while...


SVC(kernel='linear', probability=True, random_state=42)

In [12]:
# 7. Evaluate
y_pred = svm_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Cat', 'Dog']))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7374
              precision    recall  f1-score   support

         Cat       0.75      0.71      0.73      2500
         Dog       0.72      0.77      0.75      2500

    accuracy                           0.74      5000
   macro avg       0.74      0.74      0.74      5000
weighted avg       0.74      0.74      0.74      5000

Confusion Matrix:
 [[1765  735]
 [ 578 1922]]


In [13]:
# 8. Save Model
joblib.dump(svm_clf, '/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/svm_dog_cat_hog.pkl')
print("Model saved to Drive.")


Model saved to Drive.


In [19]:
# 9. Test single image
def predict_image(image_path):
    img = imread(image_path)
    img = resize(img, IMG_SIZE, anti_aliasing=True)
    if img.ndim == 3:
        img = np.mean(img, axis=2)
    feat = hog(img, **HOG_PARAMS).reshape(1, -1)
    pred = svm_clf.predict(feat)[0]
    prob = svm_clf.predict_proba(feat)[0]
    return ('Dog' if pred == 1 else 'Cat', prob)
print(predict_image('/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/test1/12500.jpg'))

('Cat', array([0.89108956, 0.10891044]))


In [ ]:
# Test single image
def predict_image(image_path):
    img = imread(image_path)
    img = resize(img, IMG_SIZE, anti_aliasing=True)
    if img.ndim == 3:
        img = np.mean(img, axis=2)
    feat = hog(img, **HOG_PARAMS).reshape(1, -1)
    pred = svm_clf.predict(feat)[0]
    prob = svm_clf.predict_proba(feat)[0]
    return ('Dog' if pred == 1 else 'Cat', prob)
print(predict_image('/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/test1/12500.jpg'))

('Cat', array([0.89108956, 0.10891044]))


In [23]:
# 9. Predict single OR multiple images
def predict_image(image_paths):
    if isinstance(image_paths, str):
        image_paths = [image_paths]  # if single path, convert to list

    results = []
    for image_path in image_paths:
        img = imread(image_path)
        img = resize(img, IMG_SIZE, anti_aliasing=True)
        if img.ndim == 3:
            img = np.mean(img, axis=2)
        feat = hog(img, **HOG_PARAMS).reshape(1, -1)
        pred = svm_clf.predict(feat)[0]
        prob = svm_clf.predict_proba(feat)[0]
        results.append({
            "image": image_path,
            "prediction": "Dog" if pred == 1 else "Cat",
            "probability_cat": prob[0],
            "probability_dog": prob[1]
        })
    return results

# 🔹 Example usage: Single image
print(predict_image('/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/test1/12500.jpg'))

# 🔹 Example usage: Multiple images
test_images = [
    '/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/test1/12413.jpg',
    '/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/test1/12412.jpg',
    '/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/test1/12352.jpg'
]

for result in predict_image(test_images):
    print(result)


[{'image': '/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/test1/12500.jpg', 'prediction': 'Cat', 'probability_cat': np.float64(0.8910895622909838), 'probability_dog': np.float64(0.1089104377090164)}]
{'image': '/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/test1/12413.jpg', 'prediction': 'Cat', 'probability_cat': np.float64(0.6719145131182706), 'probability_dog': np.float64(0.3280854868817293)}
{'image': '/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/test1/12412.jpg', 'prediction': 'Dog', 'probability_cat': np.float64(0.47849539587902135), 'probability_dog': np.float64(0.5215046041209788)}
{'image': '/content/drive/MyDrive/Colab Notebooks/Machine Learning Projects/dogs-vs-cats/test1/12352.jpg', 'prediction': 'Dog', 'probability_cat': np.float64(0.2977584883318068), 'probability_dog': np.float64(0.7022415116681934)}
